In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

n_inputs = 28 #input a row which has 28 data
max_time = 28 #28 rows
lstm_size = 100 #hidden layer node
n_classes = 10
batch_size = 50
n_batch = mnist.train.num_examples // batch_size

x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
weights = tf.Variable(tf.truncated_normal([lstm_size, n_classes],stddev=0.1))
biases = tf.Variable(tf.constant(0.1,shape=[n_classes]))

def RNN(X, weights, biases):
    inputs = tf.reshape(X, [-1,max_time,n_inputs])
    #define basic LSTM cell
#     tf.contrib.rnn.BasicLSTMCell(lstm_size)
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    #final_state[0] is cell state
    #final_state[1] is hidden state
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1],weights) + biases)
    return results

prediction = RNN(x,weights,biases)
#cost function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#train with gradient descent
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#find accuracy of trained model
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))   #convert a list of booleans into a single boolean value
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#initialize variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter" + str(epoch) + ", Testing Accuracy: " + str(acc))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Iter0, Testing Accuracy: 0.7094
Iter1, Testing Accuracy: 0.7849
Iter2, Testing Accuracy: 0.8096
Iter3, Testing Accuracy: 0.8235
Iter4, Testing Accuracy: 0.8365
Iter5, Testing Accuracy: 0.8969
